In [19]:
import os
import pandas as pd

def seccode2bsitem(str_code):
    """
    Transfer the code into b/s item of the account
    :param str_code: 6-digit code
    :return:
    str
        {
        'st': stock, 股票
        'ce': 现金及一般等价物,
        'unknown': others
        }
    """
    str_code = str_code.zfill(6)
    if str_code[:3] in ['600', '601', '603', '688', '689']:  # 未考虑B股
        return 'st'
    elif str_code[:2] in ['00', '30']:  # 未考虑B股(20,与204冲突)
        return 'st'
    elif str_code[:3] in ['204', '511', '159', '519', '521', '660']:
        return 'ce'
    elif str_code[:2] in ['13']:
        return 'ce'
    else:
        print(f'New security type found, please check {str_code} type and update the function "seccode2bsitem".')
        return 'unknown'

def process_raw_data_cash(fpath_holding):
    str_ext = os.path.splitext(fpath_holding)[1]
    if str_ext in ['.xlsx', '.xls']:
        __df_capital = pd.read_excel(fpath_holding, nrows=1)
        __df_holding = pd.read_excel(fpath_holding, skiprows=3)
    elif str_ext == '.csv':
        __df_capital = pd.read_csv(fpath_holding, nrows=1, encoding='gbk',
                                 dtype={'资产账户': str, '总资产': float, '总负债': float, '净资产': float,
                                        '资金可用金': float})
        __df_holding = pd.read_csv(fpath_holding, skiprows=3, encoding='gbk',
                                 dtype={'证券代码': str, '市值': float})
    else:
        raise TypeError('Unknown file type!')
    return __df_capital, __df_holding

In [20]:
str_date = '20200413'
fpath = f'D:/data/A_trading_data/1500+/A_result/{str_date}/2hao_MS/holding_sw.csv'
df_capital, df_holding = process_raw_data_cash(fpath)
df_holding

,记录号,证券代码,证券名称,可用数量,证券数量,成本价,最新价,盈亏,市值,资产账户,交易板块,交易账户,盈亏比例%
0,1,603360,百傲化学,12600,12600,29.79,30.99,15182.15,390474.0,2019029790,沪A,B882085314,4.05
1,2,603528,多伦科技,5000,5000,7.81,7.83,93.34,39150.0,2019029790,沪A,B882085314,0.24
2,3,603538,美诺华,10500,10500,35.30,35.35,557.04,371175.0,2019029790,沪A,B882085314,0.15
3,4,603669,灵康药业,36200,36200,10.51,11.20,25083.33,405440.0,2019029790,沪A,B882085314,6.59
4,5,603686,龙马环卫,16200,16200,15.17,15.13,-627.76,245106.0,2019029790,沪A,B882085314,-0.26
...,...,...,...,...,...,...,...,...,...,...,...,...,...
57,58,603069,海汽集团,0,6500,9.85,9.75,-618.89,63375.0,2019029790,沪A,B882085314,-0.97
58,59,002332,仙琚制药,0,24900,14.97,14.89,-1882.89,370761.0,2019029790,深A,0899171754,-0.51
59,60,002258,利尔化学,0,19500,15.19,15.14,-905.41,295230.0,2019029790,深A,0899171754,-0.31
60,61,300575,中旗股份,0,9500,29.69,29.66,-279.30,281770.0,2019029790,深A,0899171754,-0.10


In [24]:
# 此步骤之前，需要规范一下列名。
df_holding_draft = df_holding.loc[:, ['证券代码', '市值']]
df_holding_draft['bsitem'] = df_holding_draft['证券代码'].apply(seccode2bsitem)
holding_mv = df_holding_draft.groupby(by='bsitem').sum().T
dict_bs_mv = holding_mv.to_dict('record')
dict_bs_mv

,证券代码,市值,bsitem
0,603360,390474.0,st
1,603528,39150.0,st
2,603538,371175.0,st
3,603669,405440.0,st
4,603686,245106.0,st
...,...,...,...
57,603069,63375.0,st
58,002332,370761.0,st
59,002258,295230.0,st
60,300575,281770.0,st


In [31]:
df_capital.T



,0
资产账户,2019029790
总资产,4.53155e+07
市值,4.31975e+07
资金资产,2.11795e+06
资金可用金,2.11795e+06
资金冻结金,3.99109e+06
资金解冻金,4.27167e+06
货币代码,人民币
Unnamed: 8,NaN
Unnamed: 9,NaN
